# Predictions using Trained models

In [ ]:
#import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL.Image as Image
from sklearn.utils import class_weight

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Prepare the testing data

In the following cell, load the csv file which contains path for images and its labels.

In [ ]:
dataset_path = "dataset"
test_df = pd.read_csv(f"./{dataset_path}/train/labels.csv")

In [ ]:
categories = test_df.columns[1:].tolist()
print(categories)

In [ ]:
skin_tone_labels = [f"monk_{i}" for i in range(1,11)]
gender_labels = ["male", "female"]
age_labels = ["0_17", "18_30", "31_60", "61_100"]

In [ ]:
#encode test samples
test_df['skin_tone'].replace(skin_tone_labels, list(range(len(skin_tone_labels))), inplace=True)
test_df['gender'].replace(gender_labels, list(range(len(gender_labels))), inplace=True)
test_df['age'].replace(age_labels, list(range(len(age_labels))), inplace=True)

### Building the DataLoader

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, df, data_path, image_transform=None):
        self.df = df
        self.data_path = data_path
        self.image_transform = image_transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img = Image.open(f"{self.data_path}{self.df['name'][index]}")
        if self.image_transform:
            img = self.image_transform(img)
        return img

In [ ]:
image_transform = transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                         ])
#prepare the dataloader for test data
test_data = ImageDataset(test_df, f"./{dataset_path}/test/", image_transform)
test_loader = DataLoader(test_data, batch_size=256, shuffle=False)

## Build the model

In [ ]:
class ClassifierModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = torchvision.models.resnet18(pretrained=True)
        self.classifier = nn.Sequential(nn.ReLU(), nn.Linear(1000, 512), nn.ReLU(), nn.Linear(512, num_classes))
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Helper functions

In [ ]:
def make_predictions(dataloader, model):
    model.eval()
    predicted_labels = []
    with torch.no_grad():
        for imgs, _ in dataloader:
            output = model(imgs.to(device))
            preds = output.argmax(dim=1).cpu().detach().tolist()
            predicted_labels.extend(preds)
    return predicted_labels

## Make Predictions on Test data

In [ ]:
model_dir = "saved_models"

In [ ]:
#skintone_predictions
skintone_classifier = torch.load(f"./{model_dir}/skintone_classifier.pt").to(device)
skintone_predictions = make_predictions(test_loader, skintone_classifier)
del skintone_classifier

In [ ]:
#gender_predictions
gender_classifier = torch.load(f"./{model_dir}/gender_classifier.pt").to(device)
gender_predictions = make_predictions(test_loader, gender_classifier)
del gender_classifier

In [ ]:
#age_predictions
age_classifier = torch.load(f"./{model_dir}/age_classifier.pt").to(device)
age_predictions = make_predictions(test_loader, age_classifier)
del age_classifier

In [ ]:
predictions = [skintone_predictions, gender_predictions, age_predictions]

## Prepare Submission

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# calculate accuracy
acc = {}
for i in range(3):
    icat = categories[i]
    iacc = accuracy_score(test_df[icat], predictions[i])
    acc[icat] = iacc

# calculate disparity
def disparity_score(ytrue, ypred):
    cm = confusion_matrix(ytrue,ypred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    all_acc = list(cm.diagonal())
    return max(all_acc) - min(all_acc)

disp = {}
for i in range(3):
    icat = categories[i]
    idisp = disparity_score(test_df[icat], predictions[i])
    disp[icat] = idisp
disp

results = {'accuracy': acc, 'disparity': disp}
results

In [ ]:
def getScore(results):
    acc = results['accuracy']
    disp = results['disparity']
    ad = 2*acc['gender']*(1-disp['gender']) + 4*acc['age']*(1-disp['age']**2) + 10*acc['skin_tone']*(1-disp['skin_tone']**5)
    return ad

title = 'Test Submission'
    
submission = {
    'submission_name': title,
    'score': getScore(results),
    'metrics': results
}
submission

In [ ]:
import json
with open("balaji_submission.json", "w") as f:
    json.dump(submission, f, indent=4)

---